In [30]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient 

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

In [31]:
    # Connect to the CoppeliaSim server


client = RemoteAPIClient()
sim = client.require("sim")
sim.setStepping(True)

all_objects = sim.getObjectsInTree(sim.handle_scene)
for obj in all_objects:
    print(sim.getObjectName(obj))


# Handle para o ROBÔ
robotHandle = sim.getObject('Pioneer_p3dx')





DefaultCamera
XYZCameraProxy
DefaultLights
Floor
Pioneer_p3dx
DefaultXViewCamera
DefaultYViewCamera
DefaultZViewCamera
DefaultNXViewCamera
DefaultNYViewCamera
DefaultNZViewCamera
DefaultLightA
DefaultLightB
DefaultLightC
DefaultLightD
box
autoConvertedCustomizationScript_Pioneer_p3dx
Pioneer_p3dx_rightMotor
Pioneer_p3dx_leftMotor
Pioneer_p3dx_caster_freeJoint1
Pioneer_p3dx_visible
Pioneer_p3dx_connection1
Pioneer_p3dx_connection4
Pioneer_p3dx_connection2
Pioneer_p3dx_connection6
Pioneer_p3dx_connection3
Pioneer_p3dx_connection5
Pioneer_p3dx_connection9
Pioneer_p3dx_connection10
Pioneer_p3dx_connection8
Pioneer_p3dx_connection7
Pioneer_p3dx_connection11
Pioneer_p3dx_rightWheel
Pioneer_p3dx_leftWheel
Pioneer_p3dx_caster_link
Pioneer_p3dx_ultrasonicSensor1
Pioneer_p3dx_ultrasonicSensor2
Pioneer_p3dx_ultrasonicSensor3
Pioneer_p3dx_ultrasonicSensor4
Pioneer_p3dx_ultrasonicSensor5
Pioneer_p3dx_ultrasonicSensor6
Pioneer_p3dx_ultrasonicSensor7
Pioneer_p3dx_ultrasonicSensor8
Pioneer_p3dx_ultras

Exception: 1254: in sim._getObject: object does not exist, or alias/path is ill formatted.

In [22]:
try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    all_objects = sim.getObjectsInTree(sim.handle_scene)
    for obj in all_objects:
        print(sim.getObjectName(obj))


    # Handle para o ROBÔ
    robotHandle = sim.getObject('Pioneer_p3dx')
    
    # Handle para as juntas das RODAS
    l_wheel = sim.getObject('Pioneer_p3dx_leftMotor')
    r_wheel = sim.getObject('Pioneer_p3dx_rightMotor')
    
    # Dados Pioneer
    # https://www.generationrobots.com/media/Pioneer3DX-P3DX-RevA.pdf
    # L = 0.381   # Metros
    # r = 0.0975  # Metros
    
    L = 0.331
    r = 0.09751
                  
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()
    
    # Velocidade desejada (linear, angular)
    v = .2
    w = np.deg2rad(-10)

       # Cinemática Inversa
    wr = ((2.0*v) + (w*L))/(2.0*r)
    wl = ((2.0*v) - (w*L))/(2.0*r)    
    u = np.array([wr, wl])    
    
    # Enviando velocidades
    sim.setJointTargetVelocity(l_wheel, wl)
    sim.setJointTargetVelocity(r_wheel, wr)
           
    # Configuração inicial (x, y, w)
    q = np.array([0, 0, 0])
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 5:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
        
        # Cinemática Direta
        Mdir = np.array([[r*np.cos(q[2])/2, r*np.cos(q[2])/2], [r*np.sin(q[2])/2, r*np.sin(q[2])/2], [r/L, -r/L]])
        
        # Calculando posição                
        q = q + (Mdir @ u)*dt

        sim.step()


    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(l_wheel, 0)
    sim.setJointTargetVelocity(r_wheel, 0)
    
    print('CALC Pos: ', sim_time, q[:2], np.rad2deg(q[2]))
    
    pos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print('SIM Pos: ', pos)

    ori = sim.getObjectOrientation(robotHandle, sim.handle_world)
    print('SIM Ori: ', np.rad2deg(ori))
        

except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

DefaultCamera
XYZCameraProxy
DefaultLights
Floor
20cmHighWall100cm
20cmHighWall100cm0
Pioneer_p3dx
DefaultXViewCamera
DefaultYViewCamera
DefaultZViewCamera
DefaultNXViewCamera
DefaultNYViewCamera
DefaultNZViewCamera
DefaultLightA
DefaultLightB
DefaultLightC
DefaultLightD
box
20cmHighWall100cm_visible
20cmHighWall100cm_visible0
autoConvertedCustomizationScript_Pioneer_p3dx
Pioneer_p3dx_rightMotor
Pioneer_p3dx_leftMotor
Pioneer_p3dx_caster_freeJoint1
Pioneer_p3dx_visible
Pioneer_p3dx_connection1
Pioneer_p3dx_connection4
Pioneer_p3dx_connection2
Pioneer_p3dx_connection6
Pioneer_p3dx_connection3
Pioneer_p3dx_connection5
Pioneer_p3dx_connection9
Pioneer_p3dx_connection10
Pioneer_p3dx_connection8
Pioneer_p3dx_connection7
Pioneer_p3dx_connection11
Pioneer_p3dx_rightWheel
Pioneer_p3dx_leftWheel
Pioneer_p3dx_caster_link
Pioneer_p3dx_ultrasonicSensor1
Pioneer_p3dx_ultrasonicSensor2
Pioneer_p3dx_ultrasonicSensor3
Pioneer_p3dx_ultrasonicSensor4
Pioneer_p3dx_ultrasonicSensor5
Pioneer_p3dx_ultrasoni